In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [3]:
link = '/content/fake_news.csv'
fake_df = pd.read_csv(link)

In [5]:
temp = '/content/merged_dataset.csv'
true_df = pd.read_csv(temp)


In [6]:
fake_df.head(10)

,text,label
0,WARNING: A Pivotal Moment For The Stock Market...,1
1,Michele Bachmann has been pretty quiet since l...,1
2,NTEB Ads Privacy Policy NOT OVER: FBI To Condu...,1
3,Barack Obama has been out campaigning on behal...,1
4,"Terrorism Threat: Trump is Right — Profile, Pr...",1
5,Donald Trump spent Wednesday morning trying to...,1
6,"Thank God for brave people like this farmer, w...",1
7,"BAD NEWS: Hillary’s Pimping Past Exposed, Dirt...",1
8,All we can say on this one is it s about time ...,1
9,Tucker Carlson started out his show by showing...,1


In [7]:
true_df

,text,label
0,The support act slot can be a tough gig but 29...,0
1,"If you’re not a purist, you can even change th...",0
2,"Click here, external to download the BBC News ...",0
3,"Phineas Float, Jon Collyer and Lisa Stocker (l...",0
4,1 day ago Share Save Pallab Ghosh • @BBCPallab...,0
...,...,...
35274,WASHINGTON - President Donald Trump will lay ...,0
35275,CNBC reports: In the first visit to Singapore...,0
35276,PARIS - French businessman and art patron Pie...,0
35277,NAIROBI - Burundi s main opposition grouping ...,0


In [8]:
true_df.shape

(35279, 2)

In [9]:
print(true_df.isna().sum())
print(fake_df.isna().sum())

text     2
label    0
dtype: int64
text     39
label     0
dtype: int64


In [12]:
true_df.dropna(inplace=True)
fake_df.dropna(inplace=True)

In [13]:
print(true_df.isna().sum())
print(fake_df.isna().sum())

text     0
label    0
dtype: int64
text     0
label    0
dtype: int64


In [17]:
true_df.duplicated().sum()

np.int64(1393)

In [18]:

fake_df.duplicated().sum()

np.int64(6403)

In [19]:
true_df.drop_duplicates(inplace=True)
fake_df.drop_duplicates(inplace=True)

In [20]:
df = pd.concat([fake_df, true_df], axis=0, ignore_index=True)

In [21]:
df.shape

(61336, 2)

In [22]:
df

,text,label
0,WARNING: A Pivotal Moment For The Stock Market...,1
1,Michele Bachmann has been pretty quiet since l...,1
2,NTEB Ads Privacy Policy NOT OVER: FBI To Condu...,1
3,Barack Obama has been out campaigning on behal...,1
4,"Terrorism Threat: Trump is Right — Profile, Pr...",1
...,...,...
61331,WASHINGTON - President Donald Trump will lay ...,0
61332,CNBC reports: In the first visit to Singapore...,0
61333,PARIS - French businessman and art patron Pie...,0
61334,NAIROBI - Burundi s main opposition grouping ...,0


In [23]:
df.isna().sum()

,0
text,0
label,0


In [24]:
def wordcheck(text):
  text = text.lower()
  text = re.sub('\[.*?\]','',text)
  text = re.sub('\\W', " ", text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\n', '',text)
  text = re.sub('\w*\d\w*', '', text)
  return text

In [25]:
df['text'] = df['text'].apply(wordcheck)

In [26]:
df

,text,label
0,warning a pivotal moment for the stock market...,1
1,michele bachmann has been pretty quiet since l...,1
2,nteb ads privacy policy not over fbi to condu...,1
3,barack obama has been out campaigning on behal...,1
4,terrorism threat trump is right profile pr...,1
...,...,...
61331,washington president donald trump will lay ...,0
61332,cnbc reports in the first visit to singapore...,0
61333,paris french businessman and art patron pie...,0
61334,nairobi burundi s main opposition grouping ...,0


In [27]:
x = df['text']
y = df['label']


In [28]:
x_train , x_test, y_train, y_test = train_test_split(x,y, test_size=0.25)

In [30]:
y

,label
0,1
1,1
2,1
3,1
4,1
...,...
61331,0
61332,0
61333,0
61334,0


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [32]:
import pickle
pickle.dump(vectorization, open('vectorizer.pkl', 'wb'))


In [79]:
print(len(x_train))


46002


In [34]:
x_train = x_train.astype(str)


In [35]:
xv_test

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3733067 stored elements and shape (15334, 194647)>

In [36]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(xv_train, y_train)

LogisticRegression()

In [37]:
pred = model.predict(xv_test)

In [38]:
model.score(xv_test, y_test)

0.9456762749445676

In [39]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95      8476
           1       0.94      0.93      0.94      6858

    accuracy                           0.95     15334
   macro avg       0.95      0.94      0.95     15334
weighted avg       0.95      0.95      0.95     15334



In [40]:
from sklearn.tree import DecisionTreeClassifier
modeldc = DecisionTreeClassifier()
modeldc.fit(xv_train, y_train)

DecisionTreeClassifier()

In [41]:
preddc = modeldc.predict(xv_test)

In [42]:
modeldc.score(xv_test, y_test)

0.8693100299986957

In [43]:
from sklearn.ensemble import GradientBoostingClassifier
modelgc = GradientBoostingClassifier(random_state=0)
modelgc.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [44]:
predgc = modelgc.predict(xv_test)

In [45]:
modelgc.score(xv_test, y_test)

0.9105256293204643

In [46]:
from sklearn.ensemble import RandomForestClassifier
modelrf = RandomForestClassifier(random_state=0)
modelrf.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [47]:
predrf = modelrf.predict(xv_test)

In [48]:
modelrf.score(xv_test, y_test)

0.9022433807225773

In [49]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))

In [ ]:
pickel.dump(modelgc, open('model.pkl', 'wb'))

NameError: name 'pickel' is not defined

In [50]:
def output_convertor(n):
  if n == 0:
    return "Fake News"

  elif n == 1:
    return "True News"


In [51]:
def manual_testing(news):
  testing_news = {"text":[news]}
  new_def_test = pd.DataFrame(testing_news)
  new_def_test["text"] = new_def_test["text"].apply(wordcheck)
  new_x_test = new_def_test["text"]
  new_xv_test = vectorization.transform(new_x_test)
  pred_LR = model.predict(new_xv_test)
  pred_dc = modeldc.predict(new_xv_test)
  pred_gc = modelgc.predict(new_xv_test)
  pred_rf = modelrf.predict(new_xv_test)

  return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_convertor(pred_LR[0]),
                                                                                                              output_convertor(pred_dc[0]),
                                                                                                              output_convertor(pred_gc[0]),
                                                                                                              output_convertor(pred_rf[0])))

In [ ]:
news = str(input())
manual_testing(news)

GENEVA (Reuters) - North Korea and the United States accused each other on Tuesday of posing a nuclear threat, with Pyongyang s envoy declaring it would never put its atomic arsenal up for negotiation. The debate at the United Nations began when the U.S. envoy said President Donald Trump s top priority was to protect the United States and its allies against the growing threat from North Korea. To do so, he said, the country was ready to use the full range of capabilities at our disposal . U.S. Ambassador Robert Wood told the Conference on Disarmament that the path to dialogue still remains an option for Pyongyang, but that Washington was undeterred in defending against the threat North Korea poses . Fears have grown over North Korea s development of missiles and nuclear weapons since Pyongyang test-launched intercontinental ballistic missiles (ICBMs) in July. Those fears worsened after Trump warned that North Korea would face fire and fury if it threatened the United States. His remark

In [52]:
pip install newspaper3k


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 58.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 5.5 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=5b63095aa19e54742453f42e1c8462c1f027c86cad754d400731d839002f41ee
  Stored in directory: /root/.cache/pip/wheels/fc/ab/f8/cce3a9ae6d828bd346be695f7ff54612cd22b7cbd7208d68f3
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=41ca468241a87a1d399eb05f520d908e7ce72e74092093673a4bd445658d4bc3
  Stored in directory: /root/.cache/pip/wheels/80/d5/72/9cd9eccc819636436c6a6e59c22a0fb1ec

In [54]:
!pip install lxml_html_clean

In [60]:
import re
import string

In [59]:
import pickle

# Load trained vectorizer and model
vectorizer = pickle.load(open('vectorizer.pkl', 'rb'))
model = pickle.load(open('model.pkl', 'rb'))



In [68]:
def wordcheck(text):
  text = text.lower()
  text = re.sub('\[.*?\]','',text)
  text = re.sub('\\W', " ", text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\n', '',text)
  text = re.sub('\w*\d\w*', '', text)
  return text

In [67]:
from newspaper import Article

def extract_news_body_only(url):
    try:
        article = Article(url)
        article.download()
        article.parse()

        # Get the full text
        full_text = article.text.strip()

        # Option 1: Remove first line (commonly the headline)
        lines = full_text.split('\n')
        if len(lines) > 1:
            body = '\n'.join(lines[1:]).strip()
        else:
            body = full_text  # if there's only one line, return as is

        return body

    except Exception as e:
        return f"Error: {str(e)}"
s = input('enter link')
r = extract_news_body_only(s)
print(r)

enter linkhttps://www.ndtv.com/india-news/woman-faints-during-home-guard-test-gets-gang-raped-in-moving-ambulance-8954238?pfrom=home-ndtv_topscroll
The alleged assault occurred on July 24, during an ongoing Home Guard recruitment exercise at the Bihar Military Police grounds in Bodh Gaya. According to police, the woman fainted while undertaking a physical endurance test, part of the standard procedure for recruitment.

Event organisers arranged for her immediate evacuation to a hospital via an ambulance stationed at the site. The woman alleges she was raped by multiple individuals inside the ambulance while she was unconscious.

Following her statement, an FIR was registered at the Bodh Gaya police station.

An SIT and a forensic team were deployed to probe the matter. Within hours of the FIR, the SIT arrested two suspects identified as Vinay Kumar, the ambulance driver, and Ajit Kumar, the technician on board. The two men are currently in police custody and are being interrogated. Off

In [69]:
t = wordcheck(r)

In [70]:
t

'the alleged assault occurred on july   during an ongoing home guard recruitment exercise at the bihar military police grounds in bodh gaya  according to police  the woman fainted while undertaking a physical endurance test  part of the standard procedure for recruitment   event organisers arranged for her immediate evacuation to a hospital via an ambulance stationed at the site  the woman alleges she was raped by multiple individuals inside the ambulance while she was unconscious   following her statement  an fir was registered at the bodh gaya police station   an sit and a forensic team were deployed to probe the matter  within hours of the fir  the sit arrested two suspects identified as vinay kumar  the ambulance driver  and ajit kumar  the technician on board  the two men are currently in police custody and are being interrogated  officials confirmed that cctv footage from the vicinity has provided confirmation of the vehicle s route and timeline   according to the police complain

In [71]:
def predict_news(text):

    vect_text = vectorizer.transform([text])
    prediction = model.predict(vect_text)[0]

    label = 'Fake News' if prediction == 1 else 'Real News'

    return label


result = predict_news(t)
print("Prediction:", result)

Prediction: Real News


In [76]:
text = '''Shawn Helton   21st Century WireGOP presidential frontrunner Donald Trump is a populist candidate among a bevy of warhawk rivals  yet many still wonder how the real estate mogul has marched virtually unabated towards the 2016 Republican nomination.Undoubtedly, the stir Trump has caused this election season is already historic. Love him or hate him, the well-known businessman and reality TV star turned American political juggernaut is emerging as a surprising voice for US national interests in the face of an ever-increasing globalized world.Below we ll examine Trump s shocking rise in American politics, his strengths, his weaknesses and what he s up against   HAWKISH DOVE    Who s the real Donald Trump? (Photo illustration Shawn Helton of 21WIRE)The Curious Case of Donald Trump Back in 1993, brothers David and Tom Gardner founded The Motley Fool stockbroker consultancy, and at the time, it took the investment world by storm. Their mantra was simple: in business (as it is most everything else in life) timing is everything, and when a market or system becomes insular and stagnant, the conditions are ripe for a  rule breaker , and eventually, the rule breakers becomes the rule makers. This spirit of guerilla business also underpinned the Silicon Valley revolution in the 1990s and 2000s.Likewise, Donald Trump read the tea leaves and saw an opportunity in 2016.If you believe the United States is ruled by an elite establishment or is a country taken hold by statism, endless foreign intervention and crony corporatism, chances are you might be considering Donald Trump as your candidate of choice in the 2016 presidential election   and you wouldn t be alone. Similar sentiments are felt on the left-wing by supporters of Democratic Party outsider Bernie Sanders.Trump s outsider tag and anti-establishment veneer, is what seems to be resonating most among a broad base across America, as many believe that the country has been in a decline following two failed administrations in the White House under both Bush and Obama.This is the nature of populist campaigns and whether you like Trump or not, he s run one of the most effective presidential campaigns in American history, despite major backlash from the media and various political pressure groups attempting to stop his rise.The New PopulistsMuch like former Republican congressman Ron Paul in 2012, and to a lesser degree with Bernie Sanders, Trump has seemingly tapped into a cultural zeitgeist rarely seen in American politics as this year s preeminent populist choice.While Paul s staunch constitutional and libertarian beliefs propelled his popularity in 2012, it s Trump s cross-spectrum political thinking that s drawing supporters to his campaign marked by an ability to reach the average blue-collar worker, as well as successful entrepreneurs by bringing up issues concerning economic growth, immigration and global trade.To some critics however, Trump is not conservative enough, even though he s already unveiled one of the most comprehensive plans to uphold gun rights in modern history. The plan includes a proposal for nationwide concealed carry without placing a ban on certain types of guns or magazines   something which has become a hot button particularly for those on the left after a string of strange mass shooting events during much of 2015.On the other side of the political fence, many have criticized Trump s antiquated views on expanding torture techniques in the military, as well as his tough talk on internet security, privacy concerns and heavy stance on illegal immigration.The term  super-nationalistic  is being batted around to describe Trump s perspective, causing some to fear the unknown.Ironically, what has led to Trump s success has also hindered him and has arguably become a major weak point for the shrewd businessman.Trump s fiery rhetoric concerning certain issues facing America today, in some cases has worked against him, and has painted a target on him, opening the door for the media to spin anything he says, or doesn t say, or what others say about him.This is perhaps best exemplified with his tough talk on illegal immigration, along with his media counter attack against professional agitators and protesters at his events following several clashes at some of his rallies over the past few weeks.Unfortunately, his detractors and political rivals have already used these social triggers to attack the GOP frontrunner, as his Chicago rally was cancelled after erupting into a chaotic scene, leading to at least 5 arrests. Similarly, his St. Louis rally was also stormed, leading to 32 arrests, with much of the media joining in condemnation of the embattled businessman.While no one should condone the violence that results from Trump supporters clashing with outside political agitators at Trump s campaign events, critics insist that Trump has somehow escalated these volatile protests through his own incendiary language and for not using his powers of persuasion to  calm things down. Later on in this article, we ll take an even closer look at what s behind these seismic uprisings at Trump s campaign rallies.The Manhattan mogul s political story has only just begun and there are many things to consider concerning the embattled GOP frontrunner   WAR LIES    Former Secretary of State, Colin Powell and the Bush administration s findings revealed a highly coordinated disinformation campaign that led to over half a million deaths in the US-led invasion in Iraq in 2003. (Photo link foreignpolicyjournal)Alternative To War?Recently, Trump was pressed again on matters concerning the Middle East and his response has been surprisingly diplomatic with a non-interventionist platform. Trump s open condemnation of the Bush administration for the Iraq war and the Obama White House for the destruction of Libya, has resonated across the board for voters on both sides of the political spectrum. Trump has asserted that 2003 s invasion in Iraq and the blitzkrieg in Libya via NATO members in 2011, is further evidence of failed US foreign policy throughout the world.Doubling down, the GOP frontrunner Trump, has also refused to pick a side between Israel and Palestine   and in true form to his business pedigree, stating that he would rather attempt to broker a peace deal in the ages old conflict, rather than tow the party line.Calling Out  Bad  Global Trade   TRANS PACIFIC PARTNERSHIP    US leaders vote in favor of cloaked trade deal involving Asia pacific countries and America in 2015. Trump has been the only presidential candidate talking about the Trans Pacific Partnership trade deal and its vast implications on the global economy.The Trans Pacific Partnership or TPP trade deal, is something we ve covered extensively at 21WIRE over the past two years, as international trade law for new trade treaties appear to be a way for private corporations to exempt themselves from the laws of sovereign nations under the guise of a so-called free-trade agreement.As global policy makers continue to merge with the interests of multinational or transnational corporations to achieve their goals, we could view the TTP or the Transatlantic Trade and Investment Partnership (TTIP) as having zero national loyalty to any existing nation-state or its set of laws.These multi-national and transnational free trade deals have been vigorously opposed by Trump during many GOP debates this election cycle. Other critics of these deals have suggested they are being used as a way to gain a foothold on  property rights  in the future, in addition to having lasting implications on intellectual property.21WIRE previously outlined the role that the United States Trade  Representative has played in facilitating these deals between different industries: Understanding the power structure behind these draconian trade deals reveals the global chess play at work and the behind-the-scenes corporate merger s, under the watch of a biased government entity, the USTR. The USTR has been doing heavy lifting through its corporate connections with the likes of Haliburton (ESG and KBR) and Monsanto Corp, and  political insider  ties for a massive trade deal that will consolidate our future   all done outside of the democratic process.  This past week, Huffington Post published an article entitled  At Secretive Meeting, Tech CEOs And Top Republicans Commiserate, Plot To Stop Trump, : Billionaires, tech CEOs and top members of the Republican establishment flew to a private island resort off the coast of Georgia this weekend for the American Enterprise Institute s annual World Forum, according to sources familiar with the secretive gathering. The article goes on to say that the participants  couldn t help  discussing Trump   but he was not the sole purpose of the meeting. Here s some of the list of attendees as reported by HuffPo: Apple CEO Tim Cook, Google co-founder Larry Page, Napster creator and Facebook investor Sean Parker, and Tesla Motors and SpaceX honcho Elon Musk all attended. So did Senate Majority Leader Mitch McConnell (R-Ky.), political guru Karl Rove, House Speaker Paul Ryan, GOP Sens. Tom Cotton (Ark.), Cory Gardner (Colo.), Tim Scott (S.C.), Rob Portman (Ohio) and Ben Sasse (Neb.), who recently made news by saying he  cannot support Donald Trump. Recently, it was reported by Cleveland.com that  Ohio lost 112,500 jobs in 2015 resulting from the United States  trade deficit with countries that are part of the Trans-Pacific Partnership agreement, according to an analysis by the Economic Policy Institute. Additionally, the EPI s findings state that an estimated 2 million jobs have already been lost due to the TPP and that half of those were in manufacturing industry.Above all else, Trump s non-interventionist outlook on the Middle East and his diplomatic approach to Israel and Palestine, as well as his strong position on free trade deals such as the TPP and TTIP, have all contributed to placing the outspoken candidate on the political chopping block.Such is the curious case of Donald J. Trump   AMERICAN PSYCHO?    Former presidential candidate Mitt Romney propelled Trump s campaign despite his desperate appeal to the public last week in a staged pre-debate speech. (Photo link cowgernation)Political Attack DogsThe anti-Trump coalition consists of the Republican Party establishment, the Leftwing and the mainstream media   and each of these factions are presently at war with Trump and his campaign.From the onset, you could see the horror in TV broadcasters eyes when confronted by Trump   a media mogul himself who knows how to conduct a symphony on the airwaves. In many respects, he s their worst nightmare   a politician who can outplay them in their media manipulation game.For the American Leftwing, he induces revulsion, and for good reason   his ability to tame the media has enabled him to discard the political correctness which governs US media discourse.The most sustained attack however, is coming from his own party. In the September of 2015, here at 21WIRE, we discussed how neoconservatives within the Republican Party were incensed when Trump first launched his presidential campaign bid over the summer. Flashing forward to now, it s even more apparent that the media and the dying GOP party elite have attempted to shanghai Trump and the public during his unexpected political rise.Since last summer, there s been a host of media pundits and Washington figureheads looking to take on Trump in a kinda of metaphorical Thunderdome, something which has routinely played out before millions of TV viewers   and one by one, each has become politically castrated, spayed or otherwise cast aside by the outspoken tycoon.You don t have to take my word for it, just ask Meghan Kelley, Lindsay Graham, Jeb Bush, as well as former FOX News favorite and PNAC (Project for a New American Century) waving neocon, Marco Rubio   a list, that may now include Mitt Romney, following his dirty, scripted propaganda speech meant to quell Trump s primary rise.It seems this year more than any other, the American public has peeled away at the politically fixed halls of FOX, CNN and others, as mainstream networks have deliberately attempted to derail Trump through a plethora of designer media attacks, timely  gotcha  questions, ambush-style interviews and scripted debate forums working in concert to keep the apparent GOP outsider off-balance throughout the entire presidential nomination process.  Astute observers of the GOP nomination process, might conclude that many media personalities, political insiders and beltway lobbyists are comfortable with America s long ago chartered cruise towards global ambiguity and perpetual stagnation, rather than to take a chance on new blood.In fact, The Grand Old Party pet-parade nearly flatlined last week after enlisting Bain Capital businessman and former Governor of Massachusetts, Mitt Romney to unleash a scathing nationally televised assessment of Trump s presidential qualifications.Bain Capital, is an investment firm with a rich history of litigation concerning claims of disgruntled investors over profits, as well as a global company with big ties to military defense, aerospace and big agricultural giants like Monsanto.Romney s recent pre-GOP debate speech appeared to be a last-ditch effort directed by neoconservative strategists to silence Trump s campaign   but it led to a huge backlash and in the process, revealed the former governor s role as political spoiler (and thus damaged goods), as evidenced by his support of Bush-approved candidate John Kasich.Romney s speech, piggy-backed anti-Trump hashtag campaigns (#nevertrump and #stoptrump) that emerged in the weeks following an article by well-known neoconservative strategist, Peter Wehner on why he wouldn t support a Trump presidency.Here s a You Tube clip of Fox News contributor Judge Jeanine Pirro condemning Romney for awakening a sleeping giant  among the US electorate The Shape of ThingsMitt Romney and the billionaire boys club are out for blood.Trump is using big media to put out some  dangerous ideas    like the prospect for a peace deal between Israel and Palestine, and d tente with Russia, and also calling out the clear fraud of the Iraq War, as well as NATO s bombardment in Libya as Trump stated, Libya would be better off if we had left Gaddafi in power. If the politicians had gone to the beach instead, and did nothing, we d be better off in Libya and Iraq than we are today. Romney represents the financial and investment wing of the military industrial complex and an Israel first foreign policy. Along with John McCain, Romney is one of their  inside men  embedded inside the US political establishment.This is what Trump is really up against.It doesn t stop there, the dirty politricks continued this week as Rubio urged his supporters to vote for rival presidential candidate and Governor of Ohio John Kasich (a close Bush-family friend), with the Florida Senator stating, If you re a Republican primary voter in Ohio, and you don t want Donald Trump to be the nominee, John Kasich is your best bet. Incidentally, Jeb Bush met with  three non-Trump candidates  (Cruz, Rubio Kasich?) in Miami this week, a month after dropping out of the presidential race, due to his poor campaign results. The situation was described by the Wall Street Journal as the following: Former Florida Gov. Jeb Bush, who ended his 2016 candidacy last month after months of sparring with Mr. Trump, met with each of the three non-Trump candidates this week in Miami, though he hasn t offered direction on how to block Mr. Trump s path. On another interesting note, early in March, according to The Hill, CEO of FOX News, Roger Ailes, dropped his support for Rubio, if true, the move would signify this latest strategy to use Romney and Kasich as a way to undermine Trump s campaign. Here s the excerpt reported by The Hill: We re finished with Rubio,  Roger Ailes told one of the network s hosts recently, according to three unnamed sources.  We can t do the Rubio thing anymore. Flashing forward to this past week, Trump loyalist Roger Stone, wrote a revealing article entitled  Ted Cruz, A Bush By Another Name,  which detailed GOP presidential candidate Ted Cruz s close relationship to the Bush family. Here s a  passage from Stone s recent piece at The Daily Caller: Neil Bush, the son of President George H. W. Bush, who defrauded U.S. taxpayers out of $1.5 billion dollars in the savings and loan scam, and later peddled influence for the Chinese government, (who plied him with Chinese prostitutes) has formally endorsed Senator Ted Cruz for president. You can t make this stuff up. The Bush-Cruz connection is clear. Ted was George W. s brain when he ran for president. A top policy adviser, Ted maneuvered for Solicitor General in Bush World but settled for a plum at the Federal Trade Commission. Ted s a Bush man with deep ties to the political and financial establishment.  Ted and wife Heidi brag about being the first  Bush marriage    they met as Bush staffers. SEE ALSO: Ted Cruz comes up  empty  on Obamacare as government continues debt scamAs the political claws are fully out now, the recent words of Pulitzer prize nominated journo, Jon Rappoport, comes to mind: A true populist leader would accept the fact that working through usual channels, to bring about stunning changes, is a loser. The enemy is entrenched. The enemy controls the landscape. The enemy controls the machinery. Where is the populist s strength coming from? Only one place: the people who are supporting him. So he has to go back to that well, over and over again. And now he can. While pondering Trump s rivals and populism, there s a deeper agenda concerning 2016 and beyond, something which was recently pointed out here at 21WIRE.In October of 2014, The Boston Globe published a stirring article rarely seen in the mainstream entitled  Vote all you want. The secret government won t change. The Globe piece covered Tufts University political scientist Michael J. Glennon s detailed perspective on the political process in America. The article included an in-depth discussion about Glennon s book  National Security and Double Government,  a book that gives the public an inside look into the collusion between neoconservative and neoliberal political strategy on the global stage   in particular, counterterrorism policies that are routinely followed by president after president.The idea of concealed policy between neoconservatives and neoliberals was further confirmed when Robert Kagan, a co-founder and co-author of The Project for the New American Century, recently endorsed Hillary Clinton for President.This was a big story, but it barely kept the media s attention span for longer than a week.Indeed, Kagan s endorsement should provide a window for people to see the so-called (neo) conservative players behind the scenes that might have helped Clinton escape the Benghazi scandal and might step in to rescue her from her ongoing national security related email case with the FBI.Clinton s exploits in Libya are well documented and her recent endorsement by neocon strategist Kagan, is just one more glaring example of political complicity, no matter who gets elected as the president of America, the foreign policy machinations are controlled by the same operators.Coincidentally or not, Kagan also appears to be linked to Romney s anti-trump plea to the public.In a catchy call to arms, a deeply biased political plea was published on the website called War On The Rocks. The piece featured an impassioned rant from those directly connected to the military industrial complex. In fact, many war party operatives, put their seal of approval on the message including Robert Kagan, as well as Former Homeland Security secretary, Michael Chertoff, (the Chertoff Group manufactured the TSA body scanners used in most major airports) as well as over hundred other political operatives.To those aware of these tactics, the War On The Rocks Trump hit-piece, read like a foreign policy business as usual dossier, with a heavy-handed characterization of mogul, while completely misrepresenting some of Washington s most notorious military interventions in recent memory.  TRUMP CARD    Hillary Clinton and Donald Trump posters adorn Trump s Nevada caucus rally. (Photo link ulinet)Socially Engineered  Flash Mobs  Go Viral The political game changers are fully after the Trump campaign now, as the George Soros funded advocacy group MoveOn, played a large role in the Chicago University student organized shutdown at the Trump rally this past Friday, as well as a host of other splinter groups associated with Black Lives Matter (BLM) and supporters for Bernie Sanders.  BLACK BLOC   Black bloc provocateurs, have violently escalated protest movements since the 1980 s, including 2011 s Occupy Wall Street Movement. (Photo link examiner)The echoes are still felt from the Soros funded protests in Ferguson, Missouri, in 2014 and the Baltimore riots of 2015, something which immediately recalls the misguided machinations of The Occupy Movement (aka OWS) in 2011, an event that turned out to be a think-tank-spun divide and conquer stratagem designed to push naive participants into violent action.  CHICAGO CHAOS    Bernie Sanders supporters join Black Lives Matter linked groups to shutdown Trump rally. (Photo link twitter)Back in January, MoveOn publicly endorsed the Bernie Sanders campaign in what appeared to be a call to action for BLM associated groups seen backing Sanders. It s worth mentioning, that MoveOn, also supported the Obama campaign in 2008, according to the website, Sanders is the only other candidate to  threshold for an endorsement. From here on out, professional political agitator flash mobs, will be mobilized at future Trump campaign events, making it very difficult to run a normal campaign   taking away Trump s strength in numbers at his rallies. This was confirmed in Ohio recently, after a BLM activist and Sanders supporter, stormed the stage, shortly after Trump started speaking.In a Politico article entitled  Major donors consider funding Black Lives Matter,  we gain deeper insight into the political funding for BLM and the Soros linked Democratic Alliance: Major donors are usually not as radical or confrontational as activists most in touch with the pain of oppression,  said Steve Phillips, a Democracy Alliance member and significant contributor to Democratic candidates and causes. He donated to a St. Louis nonprofit group called the Organization for Black Struggle that helped organize 2014 Black Lives Matter-related protests in Ferguson, Missouri, over the police killing of a black teenager named Michael Brown. And Phillips and his wife, Democracy Alliance board member Susan Sandler, are in discussions about funding other groups involved in the movement. It worth noting Soros, a long time contributor to Hillary Clinton, donated $2 million in PAC money in 2015 and January of 2016, Soros reportedly gave $8 million to  boost  Clinton s campaign. Soros also recently funded $15 million to Latino groups looking to stop Trump s rise towards a potential GOP nomination and beyond.Any bloodshed at the hands of   vigilante mobs  at Trump rallies, should be the responsibility of those who of have sought to politicize and fund this agenda from the very beginning.Look for these political engineers, to pit choreographed conservative and liberal groups against each other at Trump s rallies or events in the future   as all of these hostile uprisings are by design.During the Days of Rage in 1969, the FBI was said to have been  tracking specific members  of the Students for a Democratic Society. The SDS was heavily connected to 60 s political agitators turned domestic terrorists, The Weather Underground. Many concluded that the group s leader, Bill Ayers, was an interloper looking for new recruits that were easily manipulated into hostile action.This begs the question: Are government agencies still infiltrating political advocacy groups to push social change?The disruptions at Trump rallies are a form of modern class warfare, one that disguises its true intentions, eroding national sovereignty through a series of upheavals, while presenting shock and awe imagery in order to manipulate the public s political perspective.Amazingly through it all, Trump s still been able to confound his opposition in the aftermath of a barrage of attacks, propelling his popularity even to even higher levels.The Ides of March primaries will be a good barometer to see what impact these chaotic clashes may have on Trump s campaign moving forward.It still remains to be seen if Trump s dynamic political vision for America will push the country further into authoritarianism or whether he can unite an embattled nation by restoring international relations throughout the world.But one thing is for sure   there s no doubt now that a systematic design is in place to unseat the GOP frontrunner and the public should question why READ MORE ELECTION NEWS AT: 21st Century Wire 2016 Files'''

In [77]:
t = wordcheck(text)

In [78]:
def predict_news(text):

    vect_text = vectorizer.transform([text])
    prediction = model.predict(vect_text)[0]

    label = 'Fake News' if prediction == 1 else 'Real News'

    return label


result = predict_news(t)
print("Prediction:", result)

Prediction: Fake News
